<a href="https://colab.research.google.com/github/codetillidie/PromotionGenie-AI/blob/main/PromotionGenie_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.4 MB/s eta 0:00:00


In [3]:
import os
import google.generativeai as genai
import torch
from PIL import Image, ImageDraw, ImageFont
from diffusers import StableDiffusionPipeline

# Configure Google Generative AI
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Define a function to upload a file to Gemini (if needed)
def upload_to_gemini(path, mime_type=None):
    file = genai.upload_file(path, mime_type=mime_type)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    return file

# Define the generation configuration for the Google model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

# Create the Google Generative model instance
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config
)

# Define the text input prompt
prompt_text = "Create copy for a banner ad for fresh green beans. Include a catchy headline, body text, and hashtags."

# Generate text for the banner using Google Generative AI
response = model.start_chat(
    history=[
        {"role": "user", "parts": [prompt_text]}
    ]
).send_message("Generate banner copy for social media")

# Parse response to use as banner_ad_data
# Assuming the response contains the needed text in separate lines
if response.text:
    banner_ad_data = {
        "copy": {
            "headline": response.text.split("\n")[0],  # First line is the headline
            "body": response.text.split("\n")[1],  # Second line is the body
            "hashtag": response.text.split("\n")[2]  # Third line is the hashtags
        },
        "image_details": {
            "prominent_color": "green",
            "suggested_background_color": ["red", "purple"]
        }
    }

# Now use Stable Diffusion from Hugging Face to generate an image
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Generate an image based on the ad copy (prompt)
prompt = banner_ad_data["copy"]["headline"] + " " + banner_ad_data["copy"]["body"]
image = pipe(prompt).images[0]  # Generate image

# Resize the image to match the banner specifications (1360px x 800px)
image = image.resize((1360, 800))

# Use PIL to draw text on the image
draw = ImageDraw.Draw(image)

# Define font and size for text rendering (make sure the font is available in Colab)
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 40)
except IOError:
    # Fallback to default font if not available
    font = ImageFont.load_default()

# Add the headline to the image
draw.text((50, 50), banner_ad_data["copy"]["headline"], font=font, fill="white")

# Add the body text
draw.text((50, 150), banner_ad_data["copy"]["body"], font=font, fill="white")

# Add hashtags
draw.text((50, 350), banner_ad_data["copy"]["hashtag"], font=font, fill="white")

# Save the image as a PNG file
output_path = "generated_banner.png"
image.save(output_path)

# Display the image
image.show()

print(f"Banner saved as {output_path}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.